### Plotly

In [1]:
import plotly.graph_objects as go
import plotly.io as pio

# Define the updated HorizonAnalytics template
HorizonAnalytics = go.layout.Template(
    layout=go.Layout(
        paper_bgcolor='#0d1b2a',  # Background color
        plot_bgcolor='#0d1b2a',  # Background color
        height=800,
        width=800 * 1.618,
        xaxis=dict(
            anchor='y',
            showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.2)',  # Softer grid lines for contrast
            tickfont=dict(
                size=36,  # Consistent with other elements
                family='Montserrat, sans-serif',
                color='#ffffff',
                weight="bold"
            ),
            title=dict(
                text='',
                font=dict(
                    size=48,  # Increase to match other elements
                    family='Montserrat, sans-serif',
                    color='#ffffff',
                    weight="bold"
                )
            ),
            linecolor='#ffffff',  # White axis lines for contrast
            linewidth=2
        ),
        yaxis=dict(
            anchor='x',
            showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.2)',  # Softer grid lines
            tickfont=dict(
                size=36,  # Consistent with x-axis
                family='Montserrat, sans-serif',
                color='#ffffff',
                weight="bold"
            ),
            title=dict(
                text='',
                font=dict(
                    size=48,  # Increase to match x-axis
                    family='Montserrat, sans-serif',
                    color='#ffffff',
                    weight="bold"
                )
            ),
            linecolor='#ffffff',  # White axis lines
            linewidth=2
        ),
        font=dict(
            color='#ffffff',  # White font for all text
            size=36,  # Uniform font size
            family='Montserrat, sans-serif',
            weight="bold"
        ),
        # Refined colorway for better visibility and differentiation
        colorway=["#FFFF00", "#33D7FF", "#A463FF", "#FFD700", 
                  "#ff4081", "#ffc107", "#00c4a0", "#a0aec0"],
        title=dict(
            text='',
            font=dict(
                size=64,  # **Big Boost in Title Size**
                color='#ffffff',
                family='Montserrat, sans-serif',
                weight="bold"
            ),
            x=0.5,  # Center title
            y=0.97  # Push title higher
        )
    ),
    data=dict(
        scatter=[
            go.Scatter(
                line=dict(width=5)  # Increased line width for better visibility
            )
        ]
    )
)

# Register the updated HorizonAnalytics template
pio.templates['HorizonAnalytics'] = HorizonAnalytics
pio.templates.default = 'HorizonAnalytics'

#### https://plotly.com/python/builtin-colorscales/
#### https://www.who.int/data/gho/data/indicators/indicator-details/GHO/prevalence-of-obesity-among-adults-bmi--30-(age-standardized-estimate)-(-)6
#### https://github.com/johan/world.geo.json
#### https://pacific-data.sprep.org/dataset/pacific-island-region-spatial-data
#### https://github.com/topojson/world-atlas/blob/master/README.md

In [2]:
import pandas as pd

# Load raw data
df_raw = pd.read_csv("f_obesity_raw.csv")

# Filter for latest year and both sexes
df_filtered = df_raw[
    (df_raw["IsLatestYear"] == True) &
    (df_raw["Dim1ValueCode"] == "SEX_BTSX")
]

# Select relevant columns and rename
d_country_obesity = df_filtered[[
    "SpatialDimValueCode",  # Country code (ISO alpha-3)
    "Location",             # Country name
    "Period",               # Year
    "FactValueNumeric"      # Obesity rate
]].rename(columns={
    "SpatialDimValueCode": "country_code",
    "Location": "country_name",
    "Period": "year",
    "FactValueNumeric": "obesity_rate"
})

# Reset index for cleanliness
d_country_obesity.reset_index(drop=True, inplace=True)

# Save to CSV
d_country_obesity.to_csv("d_country_obesity.csv", index=False)

# Optional: preview
print(d_country_obesity.head())

  country_code                           country_name  year  obesity_rate
0          SEN                                Senegal  2022         10.21
1          MOZ                             Mozambique  2022         10.26
2          LKA                              Sri Lanka  2022         10.56
3          HTI                                  Haiti  2022         10.69
4          PRK  Democratic People's Republic of Korea  2022         10.80


## tidy_country_names()

In [3]:
import pandas as pd

def tidy_country_names(csv_path):
    # Define the country name replacements
    replacements = {
        "Democratic People's Republic of Korea": "North Korea",
        "United Republic of Tanzania": "Tanzania",
        "Netherlands (Kingdom of the)": "Netherlands",
        "Venezuela (Bolivarian Republic of)": "Venezuela",
        "Iran (Islamic Republic of)": "Iran",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "Bolivia (Plurinational State of)": "Bolivia",
        "Syrian Arab Republic": "Syria",
        "occupied Palestinian territory, including east Jerusalem": "Palestine",
        "Micronesia (Federated States of)": "Micronesia",
        "Democratic Republic of the Congo": "DR Congo",
        "Republic of Korea": "South Korea",
        "Lao People's Democratic Republic": "Lao"
    }

    # Read CSV
    df = pd.read_csv(csv_path)

    # Replace country names
    df["country_name"] = df["country_name"].replace(replacements)

    # Overwrite CSV
    df.to_csv(csv_path, index=False)

    print(f"✅ Tidied and saved: {csv_path}")

tidy_country_names("d_country_obesity.csv")

✅ Tidied and saved: d_country_obesity.csv


## order_csv()

In [4]:
import pandas as pd

def order_csv(csv_path, column_name, direction="asc"):
    # Read the CSV
    df = pd.read_csv(csv_path)

    # Check sort direction
    ascending = True if direction.lower() == "asc" else False

    # Sort the DataFrame
    df_sorted = df.sort_values(by=column_name, ascending=ascending)

    # Save back to the same file
    df_sorted.to_csv(csv_path, index=False)

    print(f"✅ Sorted by '{column_name}' ({direction}) and saved: {csv_path}")

order_csv("d_country_obesity.csv", column_name="obesity_rate", direction="asc")

✅ Sorted by 'obesity_rate' (asc) and saved: d_country_obesity.csv


## load_world_geojson()

In [1]:
import os
import json
import requests

def load_world_geojson(source='local'):
    """
    Load world GeoJSON either from local file or the web.

    Parameters:
        source (str): 'local' or 'web'

    Returns:
        dict: GeoJSON data
    """
    local_path = os.path.join("..", "geo_json", "world.geo.json")

    if source == 'local':
        if os.path.exists(local_path):
            with open(local_path, "r", encoding="utf-8") as f:
                return json.load(f)
        else:
            raise FileNotFoundError(f"Local file not found at: {local_path}")
    
    elif source == 'web':
        print("Loading GeoJSON from the web...")
        geojson_url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
        response = requests.get(geojson_url)
        response.raise_for_status()
        return response.json()
    
    else:
        raise ValueError("Invalid source. Use 'local' or 'web'.")
    
world_geojson = load_world_geojson(source='web')

Loading GeoJSON from the web...


## identify_missing_countries()

In [6]:
import pandas as pd

def identify_missing_countries(geojson, df, country_column='country_code'):
    """
    Returns a list of ISO Alpha-3 country codes in the GeoJSON but missing from the DataFrame.

    Parameters:
        geojson (dict): The loaded GeoJSON dictionary.
        df (pd.DataFrame): DataFrame containing a 'country_code' column.
        country_column (str): Column name in df with ISO Alpha-3 codes (default 'country_code').

    Returns:
        missing_codes (list): ISO Alpha-3 codes not found in df.
    """
    # Extract country codes from GeoJSON
    geojson_codes = set([
        feature.get('id') or feature['properties'].get('iso_a3')
        for feature in geojson['features']
    ])
    
    # Clean nulls
    geojson_codes = {code for code in geojson_codes if code}

    # Extract codes from DataFrame
    df_codes = set(df[country_column].dropna().unique())

    # Find missing
    missing_codes = sorted(list(geojson_codes - df_codes))
    
    print(f"🧭 {len(missing_codes)} countries in GeoJSON missing from dataset:")
    for code in missing_codes:
        name = next(
            (f['properties'].get('name') or f['properties'].get('ADMIN')
             for f in geojson['features']
             if (f.get('id') or f['properties'].get('iso_a3')) == code),
            "(Unknown)"
        )
        print(f" - {code}: {name}")
    
    return missing_codes

geojson = load_world_geojson(source="web")  # "local" or "web"
df = pd.read_csv("d_country_obesity.csv")
identify_missing_countries(geojson, df)

🌍 Loading detailed GeoJSON from the web (Natural Earth)...
🧭 0 countries in GeoJSON missing from dataset:


[]

## map_frames (zoom)

In [3]:
import pandas as pd
import plotly.express as px
import os
import shutil

def map_frames(
    csv_filename,
    world_geojson,
    metric='obesity_rate',
    plot_height=1080,
    plot_width=1920,
    color_min='204,204,255',
    color_mid='102,153,255',
    color_max='0,51,204',
    parent_folder='2023-03_obesity',
    output_folder='frames_world',
    clear=True,
    zoom=None
):
    # Adjust output folder if zooming
    if zoom:
        output_folder += f"_{zoom.lower()}"

    # Load data
    df = pd.read_csv(csv_filename)
    latest_year = df['year'].max()
    df = df[df['year'] == latest_year].reset_index(drop=True)

    # Compute color scale values
    color_min_value = df[metric].min()
    color_mid_value = df[metric].mean()
    color_max_value = df[metric].max()

    geojson = world_geojson

    all_countries = pd.DataFrame({
        'country_code': [
            feature.get('id') or feature['properties'].get('iso_a3')
            for feature in geojson['features']
        ]
    })

    # Set output path
    output_path = f"/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/{parent_folder}/{output_folder}"
    if clear and os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)

    for i in range(1, len(df) + 1):
        df_partial = df.iloc[:i]
        df_merged = all_countries.merge(
            df_partial[['country_code', 'country_name', metric]],
            on='country_code', how='left'
        )

        fig = px.choropleth(
            df_merged,
            geojson=geojson,
            locations='country_code',
            color=metric,
            hover_name='country_name',
            color_continuous_scale=[
                [0.0, 'rgba(0,0,0,0)'],
                [0.00001, f'rgb({color_min})'],
                [0.5,     f'rgb({color_mid})'],
                [1.0,     f'rgb({color_max})']
            ],
            range_color=(color_min_value, color_max_value),
            scope='world',
            labels={metric: metric.replace("_", " ").capitalize()},
        )

        # Zoom logic
        if zoom == 'Europe':
            fig.update_geos(
                showcountries=True,
                countrycolor="black",
                showframe=False,
                showcoastlines=False,
                showlakes=True,
                lakecolor='#4E5D6C',
                bgcolor='rgba(0,0,0,0)',
                center={"lat": 55, "lon": 15},
                projection_scale=4
            )

        elif zoom == 'Pacific':
            fig.update_geos(
                showcountries=True,
                countrycolor="black",
                showframe=False,
                showcoastlines=False,
                showlakes=True,
                lakecolor='#4E5D6C',
                bgcolor='rgba(0,0,0,0)',
                center={"lat": -15, "lon": -165},
                projection_rotation={"lon": 160},  # rotate to keep Pacific centered
                projection_scale=2.2
    )
        else:
            fig.update_geos(
                showcountries=True,
                countrycolor="black",
                showframe=False,
                showcoastlines=False,
                showlakes=True,
                lakecolor='#4E5D6C',
                bgcolor='rgba(0,0,0,0)',
                lataxis_range=[-60, 90]
            )

        fig.update_layout(
            margin={"r": 0, "t": 0, "l": 0, "b": 0},
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            height=plot_height,
            width=plot_width,
            coloraxis_colorbar=dict(
                len=0.5,
                yanchor='middle',
                y=0.5
            )
        )

        # Save frame
        filepath = f"{output_path}/{i:04d}.png"
        fig.write_image(filepath, scale=1)
        print(f"✅ Saved {filepath}")

In [4]:
map_frames(
    csv_filename="d_country_obesity.csv",
    world_geojson=world_geojson,
    metric="obesity_rate",
    plot_height=1080,
    plot_width=1920,
    color_min="255,255,153",
    color_mid="255,165,0",
    color_max="204,0,0",
    parent_folder="2023-03_obesity",
    output_folder="frames_world",
    clear=True,
    zoom="Pacific"  # or None
)

✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0001.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0002.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0003.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0004.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0005.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0006.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0007.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_pacific/0008.png
✅ Saved /Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2023-03_obesity/frames_world_

In [26]:
target_codes = {
    'MHL', 'KIR', 'FSM', 'PYF', 'WSM', 'TUV',
    'NIU', 'COK', 'TKL', 'NRU', 'TON', 'ASM'
}

geojson_codes = {
    feature.get('id') or feature['properties'].get('iso_a3')
    for feature in world_geojson['features']
}

missing = target_codes - geojson_codes
print("Missing from GeoJSON:", missing)


Missing from GeoJSON: {'ASM', 'MHL', 'NIU', 'PYF', 'KIR', 'TKL', 'TUV', 'TON', 'COK', 'NRU', 'WSM', 'FSM'}


## text_frames (dataframe, output_folder, font_folder, font_size, font_type)

In [16]:
# from PIL import Image, ImageDraw, ImageFont
# import os
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import numpy as np

# # Directories
# base_dir = "/Users/arya/Documents/Adobe/Premiere Pro/Horizon Analytics/2025-03 afd_karte"
# output_dir = os.path.join(base_dir, "map_value_frames")  # Folder for Prozent value frames

# # Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Function to convert a percentage value to a color using the Blues colormap
# def get_map_color(value, vmin, vmax, colormap="Blues"):
#     """
#     Maps a percentage value to a color using a given colormap.
#     """
#     norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
#     cmap = plt.get_cmap(colormap)
#     rgba = cmap(norm(value))  # Get RGBA values
#     return tuple(int(c * 255) for c in rgba[:3])  # Convert to RGB (ignore alpha)

# # Function to generate frames displaying the Prozent values
# def generate_wahlkreis_value_frames(df, height=720, width=1280, 
#                                     output_dir=output_dir, font_size=51, 
#                                     font_type="ExtraBold", font_outline_width=3, 
#                                     top_margin=50):
#     """
#     Generates PNG frames for each Wahlkreis in de_2025_election using formatted 'Prozent' as text,
#     with dynamic outline color (black if <30%, white if ≥30%).
    
#     Parameters:
#         df (pd.DataFrame): The de_2025_election dataframe.
#         height (int): Image height in pixels.
#         width (int): Image width in pixels.
#         output_dir (str): Folder to save output frames.
#         font_size (int): Size of the text font.
#         font_type (str): Font weight (e.g., "Regular", "Bold", "ExtraBold").
#         font_outline_width (int): Thickness of text outline.
#         top_margin (int): Vertical space from the top edge to the text.
#     """
#     # Sort Wahlkreise by AfD support (ascending)
#     df_sorted = df.sort_values("Prozent", ascending=True).reset_index(drop=True)

#     # Find min and max Prozent for scaling colors
#     vmin, vmax = df_sorted["Prozent"].min(), df_sorted["Prozent"].max()

#     # Convert Prozent values to formatted string (xx.x%)
#     percent_labels = [f"{value:.1f}%" for value in df_sorted["Prozent"].tolist()]
    
#     # Extract corresponding colors from the Blues colormap
#     color_map = [get_map_color(value, vmin, vmax) for value in df_sorted["Prozent"].tolist()]

#     # Define Montserrat font path
#     font_path = os.path.join("..", "Montserrat", f"Montserrat-{font_type}.ttf")

#     # Try to load the selected font, fallback to default if missing
#     try:
#         font = ImageFont.truetype(font_path, font_size)
#     except IOError:
#         print(f"Montserrat font '{font_type}' not found. Using default font.")
#         font = ImageFont.load_default()

#     # Generate a frame for each Prozent value (one per row in sorted dataframe)
#     for frame_index, (percent_text, text_color, value) in enumerate(zip(percent_labels, color_map, df_sorted["Prozent"]), start=1):
#         # Determine outline color based on Prozent value
#         font_outline_color = (0, 0, 0) if value < 30 else (255, 255, 255)  # Black if <30, White if ≥30

#         # Create an empty image with transparent background
#         img = Image.new("RGBA", (width, height), color=(0, 0, 0, 0))
#         draw = ImageDraw.Draw(img)

#         # Get text size
#         text_bbox = draw.textbbox((0, 0), percent_text, font=font)
#         text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]

#         # Calculate top-aligned position
#         text_position = ((width - text_width) // 2, top_margin)  # Keep centered horizontally, align to top

#         # Add outline effect
#         if font_outline_width > 0:
#             for dx in range(-font_outline_width, font_outline_width + 1):
#                 for dy in range(-font_outline_width, font_outline_width + 1):
#                     if dx != 0 or dy != 0:  # Skip center position
#                         draw.text((text_position[0] + dx, text_position[1] + dy), percent_text, font=font, fill=font_outline_color)

#         # Draw the main text using its mapped color
#         draw.text(text_position, percent_text, font=font, fill=text_color)

#         # Construct file name (e.g., 0001.png, 0002.png)
#         file_name = f"{frame_index:06d}.png"
#         file_path = os.path.join(output_dir, file_name)

#         # Save the image
#         img.save(file_path, "PNG")

#         print(f"✅ Saved frame for: {percent_text} → {file_name} (Color: {text_color}, Outline: {font_outline_color})")

#     print(f"✅ Prozent value frames saved in: {output_dir}")

# # Run the function using de_2025_election with top-aligned text and dynamic outline colors
# generate_wahlkreis_value_frames(de_2025_election, height=720, width=1280, 
#                                 font_size=51, font_type="ExtraBold", 
#                                 font_outline_width=2, top_margin=50)


NameError: name 'de_2025_election' is not defined